# Allocation Algorithm 4

> **1.** Create child dataframe from MySQL locationdata table

> **2.** Create ei dataframe from MySQL table locationdata table

> **3.** Assign children to ei.  Children are in order based on their distance to their closest EI in descending order. After an EI has 17 children, no more children are assigned to them to create somewhat even distribution

> **4.** Push allocation to MySQL table locationdataalgo3

In [1]:
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
)
    
cursor = connection.cursor()

query = ("Select child_id_1, child.longitude, child.latitude, min(driving_distance) shortest_distance "
         "from locationdata ld "
         "join child on child.child_id = ld.child_id_1 "
         "where child_id_2 is NULL and ei_id_2 is NULL "
         "group by child_id_1 "
         "order by shortest_distance desc")
cursor.execute(query)

child_ids = cursor.fetchall()

data_list = []
for row in child_ids:
    data = {}
    data["id"] = row[0]
    data["longitude"] = row[1]
    data["latitude"] = row[2]
    child1_child_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                          "FROM locationdata "
                          f"WHERE child_id_1 = '{row[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child1_child_query)
    child1_child = cursor.fetchall()
    
    for row2 in child1_child:
        # Add columns with prefixes
        data[f'{row2[1]}_distance'] = row2[2]
        data[f'{row2[1]}_time'] = row2[3]
        
    child_child2_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                          "FROM locationdata "
                          f"WHERE child_id_2 = '{row[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child_child2_query)
    child_child2 = cursor.fetchall()
    
    for row3 in child_child2:
        data[f'{row3[0]}_distance'] = row3[2]
        data[f'{row3[0]}_time'] = row3[3]
    
    child_ei_query = ("SELECT child_id_1, ei_id_1, driving_distance, driving_time "
                      "FROM locationdata "
                      f"WHERE child_id_1 = '{row[0]}' AND child_id_2 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child_ei_query)
    child_ei = cursor.fetchall()
    
    for row4 in child_ei:
        data[f'{row4[1]}_distance'] = row4[2]
        data[f'{row4[1]}_time'] = row4[3]
        
    data_list.append(data)
        
childDataFrame = pd.DataFrame(data_list)
childDataFrame["assigned"] = ""

print(childDataFrame)

                           id  longitude   latitude  \
0    65ef076606e0a866af56d202 -81.712128  36.476418   
1    65d787f5e09e67e025e8cd71 -81.738327  36.436672   
2    65d7886ae09e67e025e8cd72 -81.738327  36.436672   
3    65d76d38e09e67e025e8cd4e -83.244423  36.348007   
4    65e5d9ad38f826f292672007 -81.751564  36.453678   
..                        ...        ...        ...   
191  65e5d71c38f826f292672001 -82.373520  36.409969   
192  65d8d6d9e09e67e025e8cdb5 -82.212234  36.589123   
193  65e5d4d238f826f292671ffc -82.374298  36.409046   
194  65e5d57838f826f292671ffd -82.374298  36.409046   
195  65d8da66e09e67e025e8cdbd -82.457680  36.297832   

     65d76430e09e67e025e8cd43_distance  65d76430e09e67e025e8cd43_time  \
0                             38.87500                        68.4574   
1                             37.45180                        65.5944   
2                             37.45180                        65.5944   
3                             76.31480         

In [3]:
connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
)
    
cursor = connection.cursor()

query = ("Select ei_id, longitude, latitude from ei ")
cursor.execute(query)

ei_ids = cursor.fetchall()

print(ei_ids)

ei_list = []
for row in ei_ids:
    data = {}
    data["id"] = row[0]
    data["longitude"] = row[1]
    data["latitude"] = row[2]
    ei_child_query = ("SELECT child_id_1, ei_id_1, driving_distance, driving_time "
                      "FROM locationdata "
                      f"WHERE ei_id_1 = '{row[0]}' AND child_id_2 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(ei_child_query)
    ei_child = cursor.fetchall()
    
    for row2 in ei_child:
        # Add columns with prefixes
        data[f'{row2[0]}_distance'] = row2[2]
        data[f'{row2[0]}_time'] = row2[3]
        
    ei1_ei_query = ("SELECT ei_id_1, ei_id_2, driving_distance, driving_time "
                    "FROM locationdata "
                    f"WHERE ei_id_1 = '{row[0]}' AND child_id_1 is NULL and child_id_2 is NULL")
    cursor.execute(ei1_ei_query)
    ei1_ei = cursor.fetchall()
    
    for row3 in ei1_ei:
        data[f'{row3[1]}_distance'] = row3[2]
        data[f'{row3[1]}_time'] = row3[3]
    
    ei_ei2_query = ("SELECT ei_id_1, ei_id_2, driving_distance, driving_time "
                    "FROM locationdata "
                    f"WHERE ei_id_2 = '{row[0]}' AND child_id_1 is NULL and child_id_2 is NULL")
    cursor.execute(ei_ei2_query)
    ei_ei2 = cursor.fetchall()
    
    for row4 in ei_ei2:
        data[f'{row4[0]}_distance'] = row4[2]
        data[f'{row4[0]}_time'] = row4[3]
        
    ei_list.append(data)

eiDataFrame = pd.DataFrame(ei_list)
eiDataFrame = eiDataFrame[eiDataFrame['id'] != '65d75e28e09e67e025e8cd3c']  #drop this EI because she is currently not seeing children
eiDataFrame["assignment_count"] = 0
print(eiDataFrame)

[('65d75a2ce09e67e025e8cd30', -82.37210845947266, 36.31948471069336), ('65d75bb0e09e67e025e8cd31', -81.977835996997, 36.722124478185), ('65d75c59e09e67e025e8cd32', -82.94459533691406, 36.197052001953125), ('65d75c87e09e67e025e8cd33', -82.4153060913086, 36.30536651611328), ('65d75cb9e09e67e025e8cd34', -82.2342300415039, 36.391361236572266), ('65d75ce1e09e67e025e8cd35', -82.5220947265625, 36.50907897949219), ('65d75d08e09e67e025e8cd36', -82.49027252197266, 36.48372268676758), ('65d75d4ce09e67e025e8cd37', -82.46031951904297, 36.29964828491211), ('65d75d80e09e67e025e8cd38', -82.20899963378906, 36.592891693115234), ('65d75da9e09e67e025e8cd39', -82.35489633044, 36.299501166818), ('65d75dd5e09e67e025e8cd3a', -82.38350677490234, 36.334129333496094), ('65d75dfae09e67e025e8cd3b', -82.54629516601562, 36.53752136230469), ('65d75e28e09e67e025e8cd3c', -82.371776729064, 36.411166263936)]
                          id  longitude   latitude  \
0   65d75a2ce09e67e025e8cd30 -82.372108  36.319485   
1   65

In [4]:
#check to make sure both dataframes have all the neccessary columns
columns_only_in_child = set(childDataFrame.columns) - set(eiDataFrame.columns)
columns_only_in_ei = set(eiDataFrame.columns) - set(childDataFrame.columns)
print(columns_only_in_child)
print("/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
print(columns_only_in_ei)

{'assigned'}
/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
{'assignment_count'}


In [5]:
#make sure both dataframes have the columns in the same order
childDataFrameOrdered = childDataFrame.sort_index(axis=1)
eiDataFrameOrdered = eiDataFrame.sort_index(axis=1)

print(childDataFrameOrdered)
print("///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
print(eiDataFrameOrdered)

     65d75a2ce09e67e025e8cd30_distance  65d75a2ce09e67e025e8cd30_time  \
0                             51.37190                        78.3826   
1                             49.97290                        75.5682   
2                             49.97290                        75.5682   
3                             65.02290                        75.1965   
4                             48.49560                        72.5964   
..                                 ...                            ...   
191                            7.84832                        15.3978   
192                           34.89640                        39.6954   
193                            7.91997                        15.8941   
194                            7.91997                        15.8941   
195                            5.75815                        12.5780   

     65d75bb0e09e67e025e8cd31_distance  65d75bb0e09e67e025e8cd31_time  \
0                              35.7864            

In [6]:
#the only null values in these dataframes should be from an entity to itself, which should always be 0
childDataFrameOrdered = childDataFrameOrdered.fillna(0)
eiDataFrameOrdered = eiDataFrameOrdered.fillna(0)

print(childDataFrameOrdered)
print("///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////")
print(eiDataFrameOrdered)

     65d75a2ce09e67e025e8cd30_distance  65d75a2ce09e67e025e8cd30_time  \
0                             51.37190                        78.3826   
1                             49.97290                        75.5682   
2                             49.97290                        75.5682   
3                             65.02290                        75.1965   
4                             48.49560                        72.5964   
..                                 ...                            ...   
191                            7.84832                        15.3978   
192                           34.89640                        39.6954   
193                            7.91997                        15.8941   
194                            7.91997                        15.8941   
195                            5.75815                        12.5780   

     65d75bb0e09e67e025e8cd31_distance  65d75bb0e09e67e025e8cd31_time  \
0                              35.7864            

In [7]:
def euclidean_distance(childRow, eiRow):
    child_values = childRow.drop(['id', 'assigned']).values
    ei_values = eiRow.drop(['id', 'assignment_count']).values
    return np.sqrt(np.sum((child_values - ei_values) ** 2))

In [8]:
childDataFrameAllocate = childDataFrameOrdered.copy()
for child_index, child_row in childDataFrameOrdered.iterrows():
    if childDataFrameAllocate.loc[child_index, "assigned"] == "":
        availableEIs = eiDataFrameOrdered[eiDataFrameOrdered['assignment_count'] < 17]
        eiDF = pd.DataFrame(columns=["id", "euclidean_distance"])
        for ei_index, ei_row in availableEIs.iterrows():
            e_value = euclidean_distance(child_row, ei_row)
            eiDF = pd.concat([eiDF, pd.DataFrame({"id": [ei_row["id"]], "euclidean_distance": [e_value]})], ignore_index=True)
        minValue = eiDF["euclidean_distance"].min()
        minIndex = eiDF["euclidean_distance"].idxmin()
        ei_with_min = eiDF.at[minIndex, "id"]

        same_location_children = childDataFrameOrdered[(childDataFrameOrdered["longitude"] == child_row["longitude"]) & 
                                                      (childDataFrameOrdered["latitude"] == child_row["latitude"])]
        childDataFrameOrdered.loc[same_location_children.index, 'assigned'] = ei_with_min
        
        rows_with_ei_with_min = childDataFrameOrdered[childDataFrameOrdered['assigned'] == ei_with_min]
        count_ei_with_min = len(rows_with_ei_with_min)
        
        ei_index = eiDataFrameOrdered[eiDataFrameOrdered['id'] == ei_with_min].index
        eiDataFrameOrdered.at[ei_index[0], 'assignment_count'] = count_ei_with_min
    else:
        print("child already assigned")
    childDataFrameAllocate = childDataFrameOrdered.copy()

print(eiDataFrameOrdered)

child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
child already assigned
    65d75a2ce09e67e025e8cd30_distance  65d75a2ce09e67e025e8cd30_time  \
0                             0.00000                        0.00000   
1                            52.02660                       57.22680   
2                            36.75820                       48.24880   
3                             3.29009                        7.91787   
4                            13.81180                       23.98970   
5                            19.90670                       25.67660   
6                            17.1875

In [9]:
for index, ei in eiDataFrameOrdered.iterrows():
    ei_id = ei["id"]
    rows_with_ei_with_min = childDataFrameOrdered[childDataFrameOrdered['assigned'] == ei_id]
    count_ei_with_min = len(rows_with_ei_with_min)
    print(f"children assigned to this ei: {count_ei_with_min}")

children assigned to this ei: 10
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 18
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 17
children assigned to this ei: 15
children assigned to this ei: 17
children assigned to this ei: 17


In [10]:
print(childDataFrameOrdered)

     65d75a2ce09e67e025e8cd30_distance  65d75a2ce09e67e025e8cd30_time  \
0                             51.37190                        78.3826   
1                             49.97290                        75.5682   
2                             49.97290                        75.5682   
3                             65.02290                        75.1965   
4                             48.49560                        72.5964   
..                                 ...                            ...   
191                            7.84832                        15.3978   
192                           34.89640                        39.6954   
193                            7.91997                        15.8941   
194                            7.91997                        15.8941   
195                            5.75815                        12.5780   

     65d75bb0e09e67e025e8cd31_distance  65d75bb0e09e67e025e8cd31_time  \
0                              35.7864            

In [11]:
for index, child in childDataFrameOrdered.iterrows():
    child_id = child["id"]
    ei_id = child["assigned"]
    updateQuery = ("Update locationdataalgo4 "
                   "SET assigned = 1 "
                   f"WHERE child_id_1 = '{child_id}' and ei_id_1 = '{ei_id}' and child_id_2 is NULL and ei_id_2 is NULL")
    cursor.execute(updateQuery)
    connection.commit()
    print("Update query pushed")

Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed
Update query pushed


In [12]:
cursor.close()
connection.close()